In [8]:
import os,sys,cv2,json
import matplotlib.pyplot as plt
import numpy as np
import albumentations as albu
import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import json
device = 'cuda:1'
sys.path.append('/rdfs/fast/home/chenyixin/')
cv2.setNumThreads(2)

In [9]:
!gpustat

node3.ib.com  Fri Dec 31 15:33:25 2021
[0] Tesla P100-PCIE-16GB | 46'C,  68 % | 13868 / 16280 MB | zhuzhimin(9081M) haitao(4773M)
[1] Tesla P100-PCIE-16GB | 33'C,   0 % |   899 / 16280 MB | haitao(887M)
[2] Tesla P100-PCIE-16GB | 37'C,   0 % | 13553 / 16280 MB | xiaotian(13543M)
[3] Tesla P100-PCIE-16GB | 63'C, 100 % | 15691 / 16280 MB | shenhuaqing(15681M)
[4] Tesla P100-PCIE-16GB | 39'C,   0 % |  7565 / 16280 MB | songzihao(7553M)
[5] Tesla P100-PCIE-16GB | 46'C,  91 % | 14549 / 16280 MB | haitao(4669M) gaoyufei(1245M) gaoyufei(8617M)
[6] Tesla P100-PCIE-16GB | 54'C,  98 % | 15691 / 16280 MB | renyike(15681M)
[7] Tesla P100-PCIE-16GB | 49'C,  93 % | 15741 / 16280 MB | zhuzhimin(15731M)


In [10]:
def get_data(png_path):
    img = cv2.imread(png_path)[...,::-1]
    return {'img':img,'septum':0,'asd':0}        

In [11]:
def postprocess(pred):
    pred_ = torch.sigmoid(pred[0,0]).detach().cpu().numpy()
    pred_ = np.uint8(pred_ * 255)
    # septum_[septum != 0] = 255
    pred_ = cv2.dilate(pred_,np.ones((3,3),dtype=np.uint8),3)
    pred_ = cv2.erode(pred_,np.ones((3,3),dtype=np.uint8),3)
    pred_ = cv2.erode(pred_,np.ones((3,3),dtype=np.uint8),3)
    pred_ = cv2.dilate(pred_,np.ones((3,3),dtype=np.uint8),3)
    pred_[pred_< 25] = 0
    pred_[pred_!= 0] = 255
    contours,_ = cv2.findContours(pred_,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    # find biggest contours
    if len(contours) == 0:
        return 0
    area = []
    for c in contours:
        area.append(cv2.contourArea(contours[0]))
    max_index = np.argmax(area)
    max_cont = contours[max_index][:,0]
    # cal biggest dis
    return np.array([np.max(max_cont[:,0]),np.min(max_cont[:,0]),np.max(max_cont[:,1]),np.min(max_cont[:,1])])
def create_box(arr):
    a,b,c,d = np.max(arr[:,0]),np.min(arr[:,1]),np.max(arr[:,2]),np.min(arr[:,3])
    width,height = a-b,c-d
    length = max(width,height) * 1
    ct_x = (a + b) / 2
    ct_y = (c+d) / 2
    a = ct_x + length / 2
    b = ct_x - length / 2    
    c = ct_y + length / 2    
    d = ct_y - length / 2    
    return a,b,c,d

In [7]:
# # create val_list:
# path = '/rdfs/data/echo/chenyixin/ASD-yixin/Train_SC-4C/'
# i = 0
# train_data_x = []
# train_data_y = []
# val_data_x = []
# val_data_y = []
# val_list = []
# for basepath,dirnames,files in os.walk(path):
#     for dirname in dirnames:
#         tmp_path = os.path.join(basepath,dirname)
#         files = os.listdir(tmp_path)
#         png_files = [os.path.join(basepath,dirname,i) for i in files if '.png' in i]
        
#         if len(png_files) == 0:
#             continue
#         if np.random.rand() > 0.8:
#             val_list.append(tmp_path)
#         i += 1
# print(val_list)
# with open('./val_list.txt','w') as f:
#     f.write('/n'.join(val_list))

In [12]:
path = '/rdfs/data/echo/chenyixin/ASD-yixin/Train_SC-4C/'
with open('./val_list.txt','r') as f:
    val_list = f.readlines()[0]
    val_list = val_list.split('/n')
i = 0
train_data_x = []
train_data_y = []
val_data_x = []
val_data_y = []
for basepath,dirnames,files in os.walk(path):
    for dirname in dirnames:
        print(dirname)
        tmp_path = os.path.join(basepath,dirname)
        files = os.listdir(tmp_path)
        png_files = [os.path.join(basepath,dirname,i) for i in files if '.png' in i]
        continue
        if len(png_files) == 0:
            continue
        png_files = sorted(png_files)
        asd_label,asd_pred = [],[]
        septum_label,septum_pred = [],[]
        imgs = []
        for png_file in png_files:
            d = get_data(png_file)
            input_shape = d['img'].shape[:2]
            img = torch.tensor(d['img']/255).float().permute(2,0,1).unsqueeze(1)
            img = F.interpolate(img,(240,320))
            img = img[:,:,:,40:280]
            imgs.append(img)
        imgs = torch.cat(imgs,dim=1)
        print('\r' + f'{i}',end='',flush=True)
        if tmp_path in val_list:
            val_data_x.append(imgs)
            if 'control' in tmp_path or '#FN' in tmp_path:
                val_data_y.append(0)
            else:
                val_data_y.append(1)
        else:
            train_data_x.append(imgs)
            if 'control' in tmp_path or '#FN' in tmp_path:
                train_data_y.append(0)
            else:
                train_data_y.append(1)
        i += 1

control
ASD
VSD__Anzhen#VsdLi_Di_Luoke#20200723083850
normal__Anzhen#NormalLi_Wangyiran#86434a60275141
VSD__Anzhen#VsdLi_Mengweiyue#20210323144735
normal__Anzhen#NormalLi_Songchangjun#470773a43d6546
VSD__Anzhen#VsdLi_Wang_Ruohan#20200923105727
normal__Anzhen#NormalLi_Majingchen#20210329154108
normal__Anzhen#NormalLi_Daihaoxuan#47deabc2fa544f
VSD__Anzhen#VsdLi_Ma_Zexuan#20200827141039
VSD__Anzhen#VsdLi_Chensian#20200622092215
normal__Anzhen#NormalLi_Chentianyi#20210331160147
VSD__Anzhen#VsdLi_Liu_Haochen#20200917083553
PDA__VSD__Anzhen#VsdPdaLi_Zhouziyi#20200908135625
normal__Anzhen#NormalLi_Chenzhongyuan#20210401143601
VSD__Anzhen#VsdLi_Wang_Yu_An#20200717084706
normal__Anzhen#NormalLi_Zhangminjia#20210412155657
normal__Anzhen#NormalLi_Wangtongwen#1b5fd9bc70be4f
VSD__Anzhen#VsdLi_Liuhao#20201102104543
VSD__Anzhen#VsdLi_Zhang_Liyuan
VSD__Anzhen#VsdLi_Baizhihua#20200831101221
normal__Anzhen#NormalLi_Wangchenyu#7e69a1e1573543
normal__Anzhen#NormalLi_Zhangyiru#71eeefdee0384f
VSD__Anzhen#Vs

IM-0015-0045.dcm#SC-4C-CDI
IM-0027-0026-0002.dcm#SC-4C-CDI
IM-0001-0024.dcm#SC-4C-CDI
IM-0001-0026.dcm#SC-4C-CDI
IM-0001-0022.dcm#SC-4C-CDI
IM-0025-0024.dcm#SC-4C-CDI
IM-0001-0074.dcm#SC-4C-CDI
IM-0001-0045.dcm#SC-4C-CDI
IM-0001-0047.dcm#best#SC-4C-CDI
IM-0005-0029.dcm#SC-4C-CDI
IM-0025-0055.dcm#SC-4C-CDI
IM-0013-0044.dcm#SC-4C-CDI
IM-0013-0040.dcm#SC-4C-CDI
IM-0013-0043-0002.dcm#SC-4C-CDI
IM-0013-0045.dcm#SC-4C-CDI
IM-0013-0046.dcm#best#SC-4C-CDI
IM-0001-0028.dcm#best#SC-4C-CDI
IM-0001-0031.dcm#SC-4C-CDI
IM-0001-0073.dcm#SC-4C-CDI
IM-0017-0032-0002.dcm#SC-4C-CDI
IM-0017-0027.dcm#best#SC-4C-CDI
IM-0001-0043.dcm#SC-4C-CDI
IM-0033-0072.dcm#SC-4C-CDI
IM-0007-0030.dcm#SC-4C-CDI
IM-0013-0025.dcm#SC-4C-CDI
IM-0005-0038.dcm#best#SC-4C-CDI
IM-0005-0039.dcm#SC-4C-CDI
IM-0015-0063.dcm#SC-4C-CDI
IM-0015-0065.dcm#SC-4C-CDI
IM-0015-0062.dcm#SC-4C-CDI
IM-0015-0052.dcm#best#SC-4C-CDI
IM-0007-0020.dcm#SC-4C-CDI
IM-0007-0033-0001.dcm#SC-4C-CDI
IM-0007-0016.dcm#best#SC-4C-CDI
IM-0001-0040.dcm#SC-4C-CDI


IM-0037-0032-0001.dcm#best#SC-4C-CDI
IM-0037-0034.dcm#SC-4C-CDI
IM-0037-0035.dcm#SC-4C-CDI
IM-0037-0032-0002.dcm#SC-4C-CDI
IM-0005-0032.dcm#SC-4C-CDI
IM-0005-0033-0002.dcm#SC-4C-CDI
IM-0005-0029.dcm#best#SC-4C-CDI
IM-0005-0030.dcm#SC-4C-CDI
IM-0003-0043.dcm#SC-4C-CDI
IM-0003-0040.dcm#best#SC-4C-CDI
IM-0003-0044.dcm#SC-4C-CDI
IM-0011-0033.dcm#SC-4C-CDI
IM-0011-0032.dcm#best#SC-4C-CDI
IM-0011-0034.dcm#SC-4C-CDI
IM-0015-0028.dcm#best#SC-4C-CDI
IM-0015-0029.dcm#SC-4C-CDI
IM-0015-0030.dcm#SC-4C-CDI
IM-0029-0060.dcm#SC-4C-CDI
IM-0011-0032.dcm#best#SC-4C-CDI
IM-0011-0034.dcm#SC-4C-CDI
IM-0003-0034.dcm#SC-4C-CDI
IM-0011-0083.dcm#SC-4C-CDI
IM-0011-0081.dcm#best#SC-4C-CDI
IM-0005-0034.dcm#SC-4C-CDI
IM-0005-0031.dcm#best#SC-4C-CDI
IM-0009-0027.dcm#SC-4C-CDI
IM-0011-0029.dcm#best#SC-4C-CDI
IM-0011-0031-0002.dcm#SC-4C-CDI
IM-0031-0029.dcm#best#SC-4C-CDI
IM-0031-0028.dcm#SC-4C-CDI
IM-0009-0036.dcm#SC-4C-CDI
IM-0015-0025.dcm#SC-4C-CDI
IM-0015-0023.dcm#SC-4C-CDI
IM-0015-0027.dcm#best#SC-4C-CDI
IM-0001

In [14]:
def data_generator(data_x,data_y,mode='train'):
    num = len(data_x)
    ite = 0
    if mode == 'train':
        max_iteration = 99999
        index = np.random.randint(num)
    if mode == 'val':
        max_iteration = len(data_x)
    while ite < max_iteration:
        if mode == 'train':
            index = np.random.randint(num)
        if mode == 'val':
            index = ite
        x = data_x[index]
        y = data_y[index]
        y = torch.tensor(y).float().unsqueeze(0).unsqueeze(0)
        if mode == 'train':
            if x.shape[1] <= 16:
                yield x,y
            else:
                start_frame = np.random.randint(x.shape[1]-16)
                x = x[:,start_frame:start_frame + 16]
                yield x,y
        if mode == 'val':
            yield x,y
        ite += 1
train_generator = data_generator(train_data_x,train_data_y,mode='train')


In [15]:
class MYMODEL(nn.Module):
    def __init__(self):
        super(MYMODEL,self).__init__()
        self.bb = torchvision.models.resnet18()
        self.bb.avgpool = nn.Identity()
        self.bb.fc = nn.Identity()
        self.bb.conv1 = nn.Conv2d(3,64,7,1,3)
        
        self.frame_max_pooling = nn.AdaptiveMaxPool2d(output_size=(1,15))
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(output_size=(1,1))
        
        self.fc = nn.Sequential(
            nn.Linear(512,512),
            nn.LeakyReLU(),
            nn.Linear(512,1)
        )
    
    def forward(self,x):
        x = x.permute(1,0,2,3)
        x = self.bb.conv1(x)
        x = self.bb.bn1(x)        
        x = self.bb.relu(x)
        x = self.bb.maxpool(x)
        
        x = self.bb.layer1(x)        
        x = self.bb.layer2(x)        
        x = self.bb.layer3(x)        
        x = self.bb.layer4(x)
        
        x = x.permute(1,2,0,3)
        x = self.frame_max_pooling(x)
        x = x.permute(2,0,1,3)
        x = self.global_avg_pooling(x)
        
        x = x.squeeze(2).squeeze(2)
        x = self.fc(x)
        return torch.sigmoid(x)

In [28]:
model = MYMODEL().to(device)
# model = torch.load('./cls_model.pth')
opt = torch.optim.Adam(model.parameters(),lr=1e-5,weight_decay=1e-4)
bce = nn.BCELoss()

In [29]:
from sklearn import metrics
def find_best_sentivity_specificity(gtA,A):
    fpr,tpr,thresholds = metrics.roc_curve(gtA,A)
#     plt.plot(fpr,tpr)
#     plt.show()
    smallest_dis = 1
    for i in range(len(fpr)):
        tmp1 = fpr[i]
        tmp2 = tpr[i]
        dis = np.power(np.power(0-tmp1,2)+np.power(1-tmp2,2),0.5)
        if smallest_dis > dis:
            smallest_dis = dis
            sen = tmp2
            spe = 1-tmp1
    return sen,spe,i
def plot_roc(gt,pred):
    auc = np.round(metrics.roc_auc_score(gt,pred),4)
    sen,spe,i = find_best_sentivity_specificity(gt,pred)
    fpr,tpr,thresholds = metrics.roc_curve(gt,pred)
    plt.plot(fpr,tpr,c= 'red',label=f'{0} ROC curve (AUROC:{0})')
    # plt.savefig('./forpaper/20211202/ROC_817_sen709_spe771.png',dpi=300)
    print(sen,spe,auc)

In [23]:
def val():
    val_generator = data_generator(val_data_x,val_data_y,mode='val')  
    LOSS = 0
    ite = 0
#     model.eval()
    ys = []
    preds = []
    for x,y in val_generator:
        x,y = x.to(device),y.to(device)
        with torch.no_grad():
            pred = model(x)
            loss = bce(pred,y)
            LOSS += loss.item() / len(val_data_x)
            print('\r' + f'{ite}/{len(val_data_x)}',end='',flush=True)
            ys.append(y.item())
            preds.append(pred.item())
            ite += 1
#     print(ys,preds)
#     plot_roc(ys,preds)
    return LOSS
val()

91/92

0.7386687453026349

In [26]:
!gpustat

node3.ib.com  Fri Dec 31 14:24:41 2021
[0] Tesla P100-PCIE-16GB | 45'C,  72 % | 15507 / 16280 MB | zhuzhimin(9081M) haitao(4773M) chenyixin(1639M)
[1] Tesla P100-PCIE-16GB | 58'C,  65 % | 15734 / 16280 MB | haitao(887M) renyike(14835M)
[2] Tesla P100-PCIE-16GB | 37'C,   0 % | 16090 / 16280 MB | chenyixin(2537M) xiaotian(13543M)
[3] Tesla P100-PCIE-16GB | 64'C,  84 % | 15691 / 16280 MB | shenhuaqing(15681M)
[4] Tesla P100-PCIE-16GB | 37'C,   0 % |    10 / 16280 MB |
[5] Tesla P100-PCIE-16GB | 44'C,  99 % | 14549 / 16280 MB | haitao(4669M) gaoyufei(1245M) gaoyufei(8617M)
[6] Tesla P100-PCIE-16GB | 58'C,  73 % | 15691 / 16280 MB | renyike(15681M)
[7] Tesla P100-PCIE-16GB | 46'C, 100 % | 15741 / 16280 MB | zhuzhimin(15731M)


In [30]:
ite = 0
best_val = val()
for x,y in train_generator:
    model.train()
    x,y = x.to(device),y.to(device)
    pred = model(x)
    loss = bce(pred,y)
    print('\r' + f'{ite},train_loss:{loss.item()}',end='',flush=True)
    
    opt.zero_grad();
    loss.backward()
    opt.step()
    
    ite += 1
    if ite % 200 == 0:
        val_loss = val()
        print(f'{ite},val_loss:{val_loss}')
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model,'./cls_model.pth')
            print('==> saved model')


91/92rain_loss:0.27246978878974915200,val_loss:0.7930289556312824
91/92rain_loss:1.03609812259674076400,val_loss:0.6673732454605079
==> saved model
400,train_loss:0.21768875420093536

/usr/local/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type MYMODEL. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


91/92rain_loss:0.218717277050018374600,val_loss:0.5786988533547391
==> saved model
91/92rain_loss:0.224917531013488777800,val_loss:0.5622818828155489
==> saved model
91/92rain_loss:0.2572389841079712341000,val_loss:0.5323170629309731
==> saved model
91/92train_loss:0.0309395715594291771200,val_loss:0.5053898488049922
==> saved model
91/92train_loss:0.1491390764713287451400,val_loss:0.6068599781315045
91/92train_loss:0.1822188049554824851600,val_loss:0.4811078078725941
==> saved model
91/92train_loss:0.02742052078247070351800,val_loss:0.48125023111187276
91/92train_loss:0.09421862661838531152000,val_loss:0.46001082736905163
==> saved model
91/92train_loss:0.89919751882553115552200,val_loss:0.44386129150402465
==> saved model
91/92train_loss:0.05295319855213165252400,val_loss:0.42655892563103076
==> saved model
91/92train_loss:0.40915042161941536762600,val_loss:0.4716465961366497
91/92train_loss:0.002704758197069168472800,val_loss:0.45991435835458105
91/92train_loss:0.1829833388328552215

KeyboardInterrupt: 